In [ ]:
from pylab import *
import pyfits,glob,time
from subprocess import call
topdir='/Users/cslage/Research/LSST/code/GUI/'
forwarddir ='/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/forward_model_varying_i'
%cd $forwarddir
import  forward
from scipy.optimize import fmin_powell


%matplotlib inline

In [ ]:
def run_sextractor(fitsfilename):
    configfile=topdir+'sextractor/30-micron-pinholes.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    minadu=10
    outname=fitsfilename

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
 
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]

    test=call(callstring)
    make_reg_from_ldac(outname+'.cat','NUMBER')



In [ ]:
thedir='/Users/cslage/Research/LSST/code/GUI/spot_sizes/'
filelist=sort(glob.glob(thedir+'bf_1.fits'))
print len(filelist)," files"


In [ ]:
tstart=time.time()
for i in range(len(filelist)): run_sextractor(filelist[i])

print time.time()-tstart

In [ ]:
print filelist

In [ ]:
# Now you should be able to check out the fits images and region files together.
# to open the first one (which is usually a good check) open the .fits and .reg like:
# ds9 *_800_*whole.fits -regions *_800_*.reg

In [ ]:
figure(figsize=(16,8))
suptitle('Spot Sizes', fontsize = 24)
for filenum, filename in enumerate(filelist):
    catname=filelist[filenum]+'.cat'
    cat=pyfits.getdata(catname,"LDAC_OBJECTS")
    x2 = cat['X2WIN_IMAGE']
    y2 = cat['Y2WIN_IMAGE']
    xy = cat['XYWIN_IMAGE']
    a = cat['A_IMAGE']
    b = cat['B_IMAGE']
    subplot(2,2,1)
    title("X2 - Mean = %.3f"%x2.mean())
    bins = 50
    hist(x2, bins = bins)
    xlabel("Second Moment (Pixels^2)", fontsize = 12)
    subplot(2,2,2)
    title("Y2 - Mean = %.3f"%y2.mean())
    bins = 50
    hist(y2, bins = bins)
    xlabel("Second Moment (Pixels^2)", fontsize = 12)
    subplot(2,2,3)
    title("A - Mean = %.3f"%a.mean())
    bins = 50
    hist(a, bins = bins)
    xlabel("Semi-Major Axis (Pixels)", fontsize = 12)
    subplot(2,2,4)
    title("B - Mean = %.3f"%b.mean())
    bins = 50
    hist(b, bins = bins)
    xlabel("Semi-Minor Axis (Pixels)", fontsize = 12)

subplots_adjust(wspace = 0.5, hspace = 0.5)

In [ ]:
atest = sqrt((x2 + y2))
print mean(atest)

In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.x2=zeros([nstamps])
        self.y2=zeros([nstamps])
        self.xy=zeros([nstamps])
        self.imax=zeros([nstamps])

def BuildSpotList(fitsfilename, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 100
    xcoomax = 1900
    ycoomin = 100
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename)
    img=pyfits.getdata(fitsfilename) 
    catname=fitsfilename[:-5]+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff

            x2sum = 0.0
            y2sum = 0.0
            xysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    x2sum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.x[i] - spotlist.xoffset[n]) * spotlist.data[i,j,n]
                    y2sum += (spotlist.y[j] - spotlist.yoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    xysum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.x2[n] = x2sum / datasum
            spotlist.y2[n] = y2sum / datasum
            spotlist.xy[n] = xysum / datasum
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.x2 = spotlist.x2[0:n]
    newspotlist.y2 = spotlist.y2[0:n]
    newspotlist.xy = spotlist.xy[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom



In [ ]:
spotlist = BuildSpotList(filelist[0], 8000, 9, 9, 1.3, 1.7)
param0 = [1.0, 1.0]
args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
print spotlist.nstamps
print Result


In [ ]:
print spotlist.nstamps
imshow(spotlist.data[:,:,33],interpolation='Nearest')

In [ ]:
# So this agrees perfectly.  I created a GalSim image with a pixel size of 0.2" and a spot sigma of 0.3", and the forward modeling returned a sigma of 1.500 pixels.  All in agreement.  A and B are also 1.5 pixels. As for second moments, they should each be about 2.33 (1.5^2 + some pixelization error)  They are half this?????                  Also a^2 = sqrt(x2 + y2), not sqrt((x2 + y2) / 2) like the documentation says.

In [ ]:
figure(figsize=(16,8))

subplot(2,2,1)
title("X2 - Mean = %.3f"%spotlist.x2.mean())
bins = 50
hist(spotlist.x2, bins = bins)
xlabel("Second Moment (Pixels^2)", fontsize = 12)
subplot(2,2,2)
title("Y2 - Mean = %.3f"%spotlist.y2.mean())
bins = 50
hist(spotlist.y2, bins = bins)
xlabel("Second Moment (Pixels^2)", fontsize = 12)
subplot(2,2,3)
title("XY - Mean = %.3f"%spotlist.xy.mean())
bins = 50
hist(spotlist.xy, bins = bins)
xlabel("Second Moment (Pixels^2)", fontsize = 12)

subplots_adjust(wspace = 0.5, hspace = 0.5)